In [ ]:
!pip install -q roboflow
!pip install -q unzip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
!pip install -e .


In [ ]:
!yolo task=detect mode=train model=yolov8n.pt data=/content/data.yaml epochs=50 imgsz=640 batch=16

In [ ]:
!curl -L "https://app.roboflow.com/ds/7av46qCf8I?key=plNn7bS7Ef" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cd /content
!wget http://14.225.204.171:9998/Self_trainning_3.zip

In [ ]:
%cd /content
!unzip /content/Self_trainning_3.zip

In [ ]:
# Train the model
!yolo task=detect mode=train model=/content/best.pt data=/content/Self_trainning/data.yaml epochs=10 imgsz=640 batch=32

import os
import glob
from google.colab import files

# Define the base path where YOLO saves the training results
base_path = "/content/ultralytics/runs/detect/"

# Get a list of all training folders
train_folders = glob.glob(os.path.join(base_path, 'train*'))

# Find the most recently modified training folder
latest_train_folder = max(train_folders, key=os.path.getmtime)

# Construct the path to the best model
model_path = os.path.join(latest_train_folder, "weights", "best.pt")

# Check if the model file exists and download it
if os.path.exists(model_path):
    print(f"Training completed. Model found in {latest_train_folder}. Preparing to download...")
    files.download(model_path)
else:
    print("Model not found. Please check the training process and paths.")


In [ ]:
%cd /content
!wget http://14.225.204.171:9998/validate_data_2.zip
!unzip /content/validate_data_2.zip


In [ ]:
%cd /content
!wget http://14.225.204.171:9998/best_0827.pt

/content
--2024-09-09 03:20:26--  http://14.225.204.171:9998/best_0827.pt
Connecting to 14.225.204.171:9998... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18526742 (18M) [application/octet-stream]
Saving to: ‘best_0827.pt’

best_0827.pt        100%[===================>]  17.67M  2.04MB/s    in 24s     

2024-09-09 03:20:51 (743 KB/s) - ‘best_0827.pt’ saved [18526742/18526742]



In [ ]:
import cv2
import os
import pandas as pd
from ultralytics import YOLO
from google.colab.patches import cv2_imshow  # Import for displaying images in Colab

# Define paths
model_path = '/content/best_0827.pt'  # Path to your trained model
input_folder = '/content/validate/images'           # Path to the folder containing images
label_folder = '/content/validate/labels'           # Path to the folder containing ground truth labels
output_folder = '/content/output'          # Path to save the output images
csv_output_path = '/content/evaluation_results.csv'  # Path to save the evaluation results CSV
txt_output_path = '/content/final_conclusion.txt'    # Path to save the final conclusion TXT

# Create output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the YOLOv8 model
model = YOLO(model_path)

# Initialize lists to store evaluation results
results_data = []

# Function to parse label file into a list of ground truth classes
def parse_label_file(label_path):
    classes = []
    with open(label_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            class_id = parts[0]
            # Map class IDs to class names if necessary
            class_name = {
                '5': 'SpeedLimit_50',
                '6': 'SpeedLimit_60',
                '7': 'SpeedLimit_70',
                '8': 'SpeedLimit_80',
                '9': 'SpeedLimit_90'
            }.get(class_id, class_id)  # Default to class_id if not found
            classes.append(class_name)
    return classes

# Iterate over all images in the input folder
for image_name in os.listdir(input_folder):
    # Full path to the image
    image_path = os.path.join(input_folder, image_name)

    # Check if the file is an image
    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    # Perform detection
    results = model(image_path)

    # Extract detected class names
    detected_classes = [model.names[int(cls)] for cls in results[0].boxes.cls]

    # Corresponding label file
    label_file = image_name.replace('.jpg', '.txt')  # Adjust extension if necessary
    label_path = os.path.join(label_folder, label_file)

    # Parse ground truth classes from label file
    if os.path.exists(label_path):
        gt_classes = parse_label_file(label_path)
    else:
        gt_classes = []

    # Compare detections with ground truth classes
    tp = len(set(detected_classes) & set(gt_classes))  # True Positives: correct class detections
    fp = len(detected_classes) - tp  # False Positives: incorrect class detections
    fn = len(gt_classes) - tp  # False Negatives: missed ground truth classes

    # Save results
    results_data.append({
        'Image': image_name,
        'True Positives': tp,
        'False Positives': fp,
        'False Negatives': fn,
        'Detected Classes': detected_classes,
        'Ground Truth Classes': gt_classes
    })

    # Save the output image with detections
    detected_img = results[0].plot()
    output_path = os.path.join(output_folder, f"detected_{image_name}")
    cv2.imwrite(output_path, detected_img)
    cv2_imshow(detected_img)  # Display the image in Colab

# Convert results to a DataFrame and save to CSV
df = pd.DataFrame(results_data)
df['Precision'] = df['True Positives'] / (df['True Positives'] + df['False Positives'])
df['Recall'] = df['True Positives'] / (df['True Positives'] + df['False Negatives'])
df['F1 Score'] = 2 * (df['Precision'] * df['Recall']) / (df['Precision'] + df['Recall'])

# Save evaluation results to CSV
df.to_csv(csv_output_path, index=False)

# Calculate overall evaluation metrics
total_tp = df['True Positives'].sum()
total_fp = df['False Positives'].sum()
total_fn = df['False Negatives'].sum()

overall_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
overall_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
overall_f1 = 2 * (overall_precision * overall_recall) / (overall_precision + overall_recall) if (overall_precision + overall_recall) > 0 else 0

# Save final conclusion to TXT file
with open(txt_output_path, 'w') as f:
    f.write(f"Overall Evaluation Metrics:\n")
    f.write(f"Total True Positives: {total_tp}\n")
    f.write(f"Total False Positives: {total_fp}\n")
    f.write(f"Total False Negatives: {total_fn}\n")
    f.write(f"Overall Precision: {overall_precision:.2f}\n")
    f.write(f"Overall Recall: {overall_recall:.2f}\n")
    f.write(f"Overall F1 Score: {overall_f1:.2f}\n")

print("Evaluation completed and results saved to:", csv_output_path)
print("Final conclusion saved to:", txt_output_path)


In [22]:
!rm -r /content/validate